In [ ]:
from planet4.dbscan import DBScanner

from planet4 import io, clustering, plotting, markings, dbscan
import seaborn as sns
sns.set_context('notebook')
blotchcols = markings.Blotch.to_average
fancols = markings.Fan.to_average

In [ ]:
%matplotlib nbagg

In [3]:
import socket
if socket.gethostname().startswith('macd2860'):
    %config InlineBackend.figure_format = 'retina'

In [ ]:
%config InlineBackend.figure_format = 'png'

In [5]:
# At the beginning of the notebook
import logging
import sys
logger = logging.getLogger('planet4.dbscan')
ch = logging.StreamHandler(sys.stderr)
# fmt = logging.Formatter('%(name)s - %(message)s')
fmt = logging.Formatter('%(message)s')
ch.setFormatter(fmt)
logger.addHandler(ch)

In [ ]:
logger.setLevel(logging.DEBUG)

In [ ]:
logger.setLevel(logging.INFO)

In [4]:
def cluster_and_plot(image_id,
                     eps=15,
                     dynamic=True,
                     delta_fan_angle=20,
                     delta_blotch_angle=20,
                     dir_ext='_check2',
                     msf=0.15,
                     data=None,
                     production=False,
                     min_samples=3,
                     radii=False,
                     dbscan=True,
                     hdbscan_diff=0,
                     figtitle='',
                     prob=0.1,
                     only_core=True):
    from planet4 import plotting, clustering
    cm = clustering.ClusteringManager(
        do_dynamic_min_samples=dynamic,
        output_dir='sequential_clustering' + dir_ext,
        min_samples_factor=msf,
        use_DBSCAN=dbscan,
        eps=eps,
        eps_fanangle=delta_fan_angle,
        eps_blotchangle=delta_blotch_angle,
        min_samples=min_samples,
        include_radii=radii,
        hdbscan_min_samples_diff=hdbscan_diff,
        proba_cut=prob,
        only_core=only_core)

    cm.cluster_image_id(image_id, data=data)
    plotting.plot_image_id_pipeline(
        image_id, save=True, savetitle=figtitle, cm=cm, with_center=True,
        figtitle=figtitle)
    if production:
        plt.close('all')
        return dict(id_=image_id)
    else:
        return cm

In [ ]:
def create_and_save_randoms():
    myids = np.random.choice(ids, 100)
    np.save('myids.npy', myids)

myids = np.load('myids.npy')

len(myids)

from nbtools import execute_in_parallel

results = execute_in_parallel(cluster_and_plot, myids)

Here's my comments from the review"

APF0000br5 - seems like the big blotch should have been seen

APF0000bu5 - seems like middle fan should be there - seems too strict a cut not clustering issue?

APF0000ek1- yellow final blotch comes out of no where

APF0000pbr - bottom right blotch seems like it should have survived

APF00001dt - cyan fan seems bigger than it should be


In [ ]:
ids = 'br5 bu5 ek1 pbr 1dt 1dr 1fe dch bvc 1c5 1ab 1dk 18s 1b0 1cl 1ct 1at 1al 1aa 10p 185 139 13t 15k 17a'.split()

In [ ]:
for id_ in ids:
    print(id_)
    for kind in ['fan','blotch']:
        print(kind)
        dbscanner = DBScanner(id_)
        dbscanner.parameter_scan(kind, [0.1, 0.15], [10, 15, 20],
                                 with_radii=True, with_angles=True)
        plt.close('all')

In [ ]:
db = DBScanner('br5')

In [ ]:
db.show_markings()

In [ ]:
from shapely.geometry import Point

In [ ]:
p1 = Point(1,2)
p2 = Point(10, 11)

In [ ]:
p1.distance??

In [ ]:
plt.close('all')

In [ ]:
db.cluster_and_plot('blotch', 10, 5, with_angles=True, with_radii=True, eps_large=40)

In [ ]:
db.parameter_scan('blotch', [0.1,0.15], [10, 15, 20])

In [ ]:
%debug

In [ ]:
sizes = []
for _,b in blotches.iterrows():
    B = markings.Blotch(b, scope='planet4')
    sizes.append(B.area)

In [ ]:
%matplotlib nbagg

In [ ]:
plt.figure()
plt.hist(sizes, bins=50);

In [ ]:
db.parameter_scan('fan', [0.1,0.15], [10, 15, 20])

In [ ]:
db.cluster_and_plot('blotch', 20, 3)
ax = plt.gca()
ax.get_title()

In [ ]:
db.parameter_scan('fan', [0.07, 0.1, 0.15], [15,20])

In [ ]:
db.parameter_scan('blotch', [0.07, 0.1, 0.15], [15,20])

In [ ]:
ek1.cluster_and_plot('blotch', 20, 3)

In [ ]:
ek1.p4id.plot_blotches(data=ek1.finalclusters[5])

In [ ]:
ek1.p4id.plot_blotches(data=ek1.averaged[5])

In [ ]:
p4id = markings.ImageID('1fe', scope='planet4')
blotches = p4id.get_blotches()

In [ ]:
X = blotches['x y'.split()]

In [ ]:
dbscanner = DBScanner(X, min_samples=5, eps=20)

In [ ]:
clusters = [blotches.loc[idx] for idx in dbscanner.clustered_indices]

In [ ]:
from planet4.clustering import cluster_angles

In [ ]:
bucket = []
for cluster in clusters:
    print(cluster.shape)
    bucket.append([cluster.loc[idx] for idx in cluster_angles(cluster, 'blotch', 5)])

In [ ]:
for item in bucket:
    for subitem in item:
        print(subitem.shape)

In [ ]:
        cluster_and_plot('1dr', production=True, dynamic=True,
                         msf=msf, eps=eps, radii=False, dbscan=True,
                         figtitle=figtitle)

In [ ]:
cm = cluster_and_plot('1dt', production=False, msf=0.1, dynamic=True,
                      radii=False, dbscan=False)

In [ ]:
df = pd.read_csv('fuckdf.csv')

In [ ]:
(df - df.mean(axis=0))/df.std(axis=0)

In [ ]:
df[df.apply(lambda x: np.abs(x - x.mean()) / x.std() < 1).all(axis=0)]

In [ ]:
from scipy.stats import zscore

In [ ]:
zscore??

In [ ]:
pd.DataFrame(zscore(df,ddof=1))

In [ ]:
def highlight_bigger_std(x):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_true = (np.abs(x - x.mean()) / x.std() > 2)
    return ['background-color: yellow' if v else '' for v in is_true]
#     return is_true

In [ ]:
df.style.apply(highlight_bigger_std)

In [ ]:
cm = cluster_and_plot('pbr', production=False, msf=0.1, dynamic=True,
                      radii=False)

In [ ]:
cm = cluster_and_plot('pbr',eps=20, production=False, msf=0.1, dynamic=True,
                      radii=True)

In [ ]:
cm.db

In [ ]:
imgid = '1at'
imgid = 'dch'
imgid = 'bvc'
imgid = '1dr'
imgid = '1fe'
imgid = 'br5'
imgid = 'ek1'
p4id = markings.ImageID(imgid, scope='planet4')

In [ ]:
data = p4id.get_blotches()

In [ ]:
from planet4.dbscan import DBScanner

In [ ]:
current_X = data[['x','y']].values

In [ ]:
clusterer = DBScanner(current_X, eps=15, min_samples=3)

In [ ]:
clusterer.n_clusters_

In [ ]:
cluster = data.loc[clusterer.clustered_indices[0]]

p4id.plot_blotches(blotches=cluster,with_center=True)

In [ ]:
cluster[blotchcols]

In [ ]:
indices = clustering.cluster_angles(cluster, 'blotch', eps_blotchangle=10)
indices

In [ ]:
angle_cluster_data = cluster.loc[indices[0], blotchcols +['user_name']]

In [ ]:
angle_cluster_data

In [ ]:
df = angle_cluster_data[blotchcols]

In [ ]:
df[df.apply(lambda x: np.abs(x - x.mean()) / x.std() < 1).all(axis=1)]

In [ ]:
clustering.get_average_object(angle_cluster_data[blotchcols], 'blotch')

In [ ]:
p4id.plot_blotches(blotches=cluster.loc[indices[0]], with_center=True)

In [ ]:
df = cluster.loc[indices[0]][blotchcols]

In [ ]:
df['area'] = df.apply(lambda x: np.pi*x.radius_1*x.radius_2, axis=1)

In [ ]:
df

In [ ]:
col='radius_1'

In [ ]:
df.radius_1.std()

In [ ]:
df[np.abs(df[col]-df[col].mean())<=(1*df[col].std())]

In [ ]:
df[df.apply(lambda x: np.abs(x - x.mean()) / x.std() < 1).all(axis=1)]

In [ ]:
subclus

In [ ]:
testblotch = markings.Blotch?

In [ ]:
testblotchdata = dict(x=340, y=340, angle=127, radius_1=250, radius_2=186)

In [ ]:
testblotch = markings.Blotch(
    pd.DataFrame(
        testblotchdata, index=[0]), scope='planet4')
fig, ax = plt.subplots()
ax.add_artist(testblotch)
ax.set_xlim(0, 800)
ax.set_ylim(0, 600)

In [ ]:
testblotch = markings.Blotch(
    pd.DataFrame(testblotchdata, index=[0]),
    scope='planet4')

p4id.plot_blotches(blotches=[testblotch])

In [ ]:
from sklearn.cluster import DBSCAN


class DBScanner(object):
    """Execute clustering and create mean cluster markings.

    The instantiated object will execute:

        * _run_DBSCAN() to perform the clustering itself
        * _post_analysis() to create mean markings from the clustering results


    Parameters
    ----------
    current_X : numpy.array
        array holding the data to be clustered, preprocessed in ClusterManager
    eps : int, optional
        Distance criterion for DBSCAN algorithm. Samples further away than this value don't
        become members of the currently considered cluster. Default: 10
    min_samples : int, optional
        Mininum number of samples required for a cluster to be created. Default: 3
    """

    def __init__(self, X, eps=15, min_samples=3, only_core=False):
        self.X = X
        self.eps = eps
        self.min_samples = min_samples
        self.only_core = only_core

        # these lines execute the clustering
        self._run_DBSCAN()

    def _run_DBSCAN(self):
        """Perform the DBSCAN clustering."""
        db = DBSCAN(self.eps, self.min_samples).fit(self.X)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True

        labels = db.labels_
        unique_labels = set(labels)
        colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))

        self.n_clusters_ = len(unique_labels) - (1 if -1 in labels else 0)

        self.clustered_indices = []  # list of `kind` cluster average objects
        self.n_rejected = 0
        # loop over unique labels.
        for k, col in zip(unique_labels, colors):
            # get indices for members of this cluster
            class_member_mask = (labels == k)
            if self.only_core:
                cluster_members = (class_member_mask & core_samples_mask)
            else:
                cluster_members = class_member_mask

            if k == -1:
                col = 'black'
                self.n_rejected = len(cluster_members)
            else:
                xy = self.X[cluster_members]
                if xy.shape[1] > 1:
                    y = xy[:, 1]
                else:
                    y = [0] * xy.shape[0]
                plt.plot(
                    xy[:, 0],
                    y,
                    'o',
                    markerfacecolor=col,
                    markeredgecolor='black',
                    markersize=14)
                xy = self.X[class_member_mask & ~core_samples_mask]
                if xy.shape[1] > 1:
                    y = xy[:, 1]
                else:
                    y = [0] * xy.shape[0]
                plt.plot(
                    xy[:, 0],
                    y,
                    'o',
                    markerfacecolor=col,
                    markeredgecolor='black',
                    markersize=6)
                self.clustered_indices.append(cluster_members)
        plt.gca().invert_yaxis()
        plt.title('Estimated number of clusters: %d' % self.n_clusters_)
        self.db = db

In [ ]:
cluster[blotchcols]

In [ ]:
xy_angles = clustering.angle_to_xy(cluster.angle, 'blotch')

In [ ]:
xy_angles

In [ ]:
xy_angles.shape

In [ ]:
plt.figure(figsize=(5*1.3,5))
clusterer = DBScanner(xy_angles, eps=20*np.pi/360, min_samples=3)

In [ ]:
data.loc[clusterer.clustered_indices[1]]

In [ ]:
for cluster_members in clusterer.clustered_indices:
    clusterdata = data.loc[cluster_members, blotchcols + ['user_name']]
    print(len(clusterdata))
    angle_clustered = clustering.cluster_angles(clusterdata, 'blotch')
    for indices in angle_clustered:
        angle_clusterdata = clusterdata.loc[indices, blotchcols +
                                            ['user_name']]
        filtered = angle_clusterdata.groupby('user_name').first()
        print(len(filtered))

In [ ]:
cm.min_samples

In [ ]:
30* cm.min_samples_factor

In [ ]:
cm.reduced_data['blotch']

In [ ]:
cm.cluster_angles

In [ ]:
db = clustering.cluster_angles(cluster, 'blotch')
len(db[0])

In [ ]:
len(cluster)

In [ ]:
filtered = cluster.groupby('user_name').first()

In [ ]:
plt.figure()
filtered.angle.hist()

In [ ]:
toprint = cluster2[markings.Fan.to_average + ['user_name', 'marking', 'classification_id']]

In [ ]:
toprint.to_clipboard(index=False)

In [ ]:
def add_angle_vector(df):
    new = df.copy()
    new['xang'] = np.cos(np.deg2rad(df.angle))
    new['yang'] = np.sin(np.deg2rad(df.angle))
    return new

In [ ]:
cluster2 = add_angle_vector(cluster2)

In [ ]:
cluster2


# testing angle deltas

In [ ]:
def angle_to_xy(angle):
    x = np.cos(np.deg2rad(angle))
    y = np.sin(np.deg2rad(angle))
    return np.vstack([x,y]).T

In [ ]:
def cluster_angles(angles, delta_angle):
    dist_per_degree = 0.017453070996747883
    X = angle_to_xy(angles)
    clusterer = DBScanner(X, eps=delta_angle*dist_per_degree, min_samples=3)
    return clusterer

In [ ]:
clusterer = cluster_angles(cluster.angle, 10)

In [ ]:
clusterer.db.core_sample_indices_

In [ ]:
clusterer.db.labels_

In [ ]:
cluster.shape

In [ ]:
clusterer.clustered_indices

In [ ]:
cluster2.iloc[clusterer.clustered_data[0]]

In [ ]:
dbscanner.reduced_data[0]

this means all ellipses were clustered together. eps=10 picks 3 out of these 6.

In [ ]:
clusterdata = data.iloc[dbscanner.reduced_data[0]]

so clusterdata is just the same as the input data, i just repeat the exact same code steps here for consistency.

In [ ]:
clusterdata[blotchcols]

In [ ]:
meandata = clusterdata.mean()
meandata

In [ ]:
from scipy.stats import circmean

In [ ]:
meandata.angle = circmean(clusterdata.angle, high=180)

In [ ]:
meandata

In [ ]:
n_class_old = data.classification_id.nunique()
n_class_old

In [ ]:
# number of classifications that include fan and blotches
f1 = data.marking == 'fan'
f2 = data.marking == 'blotch'
n_class_fb = data[f1 | f2].classification_id.nunique()
n_class_fb

In [ ]:
data=data[data.marking=='blotch']

In [ ]:
plotting.plot_raw_blotches('bvc')

In [ ]:
fans.plot(kind='scatter', x='x',y='y')
plt.gca().invert_yaxis()

In [ ]:
fx1 = data.x < 400 
fx2 = data.x > 300
fy1 = data.y_R > 300
fy2 = data.y_R < 400

In [ ]:
data = data.reset_index()

In [ ]:
data[fx1 & fx2 & fy1 & fy2].angle

In [ ]:
cm.dbscanner.reduced_data

# testing cluster_image_name

In [ ]:
cm = clustering.ClusteringManager()

In [ ]:
cm.cluster_image_name('PSP_003092_0985')